In [1]:
# %% Cell 1: dodaj ścieżkę do folderu wyżej, jeśli notebook jest w pricing_methods/
import sys, os
sys.path.insert(0, os.path.abspath('..'))


In [2]:
import numpy as np
import pandas as pd
from pricing_methods.data_structures import ModelParams
from pricing_methods.stochastic_mesh_pricer import StochasticMeshPricer
from stochastic_mesh                        import simulate_gbm_paths, price_option

In [11]:

# Jednowymiarowe parametry
spot_price     = [100.0]
drift          = [0.05]
vol            = [0.2]
rf_rate        = 0.05
dividend_yield = [0.0]
maturity       = 1.0
n_steps        = 10
n_paths_mesh   = 10  # liczba ścieżek w siatce
n_paths_est    = 1000   # liczba ścieżek do estymacji dolnego ograniczenia
dt             = maturity / n_steps
K              = 100.0

# funkcja wypłaty — stara implementacja (d, b, m) → (b, m)
def payoff_old(X: np.ndarray) -> np.ndarray:
    # X[0] ma kształt (b, m)
    return np.maximum(X[0] - K, 0)

# funkcja wypłaty — nowa klasa (N, M+1, d) → (N, M+1)
def payoff_new(paths: np.ndarray) -> np.ndarray:
    return np.maximum(paths[..., 0] - K, 0)

# --- stara implementacja mesh ---
asset_price_mesh = simulate_gbm_paths(
    spot_price,
    drift,
    vol,
    n_steps,
    n_paths_mesh,
    maturity
)
low_price, high_price = price_option(
    asset_price_mesh,
    payoff_old,
    spot_price,
    drift,
    vol,
    rf_rate,
    dividend_yield,
    maturity,
    dt,
    n_paths=n_paths_est
)
print(f"Stochastic mesh (stara wersja, 1D): low = {low_price:.4f}, high = {high_price:.4f}")


Stochastic mesh (stara wersja, 1D): low = 4.0450, high = 3.0272


In [12]:

# --- Parametry jednowymiarowe (d = 1) ---
spot_price     = [100.0]                   # S₀
vol            = [0.2]                     # σ
corr_matrix    = np.array([[1.0]])         # korelacja (tu tylko 1×1)
dividend_yield = [0.0]
rf_rate        = 0.05
maturity       = 1.0
n_steps        = 10
n_paths_est    = 1000
K              = 100.0

# --- Funkcja wypłaty ---
payoff_fn = lambda paths: np.maximum(paths[..., 0] - K, 0)

# --- ModelParams ---
params = ModelParams(
    spot           = np.array(spot_price),
    rf_rate        = rf_rate,
    sigma          = np.array(vol),
    corr           = corr_matrix,
    dividend_yield = np.array(dividend_yield),
    maturity       = maturity,
    n_steps        = n_steps,
    n_paths        = n_paths_est
)

# --- Inicjalizacja pricer’a z n_paths_mesh=10 ---
mesh_pricer = StochasticMeshPricer(
    model_params   = params,
    payoff         = payoff_fn,
    antithetic     = False,
    seed           = 42,
    n_paths_mesh   = n_paths_mesh    # liczba ścieżek w siatce
)

# --- Wywołanie metody price() ---
low_price, high_price = mesh_pricer.price()

print(f"low  = {low_price:.4f}")
print(f"high = {high_price:.4f}")




low  = 7.1687
high = 3.8417
